# Import

In [25]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from keras_preprocessing.sequence import pad_sequences
from varname import nameof
from numpy import savetxt
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import wget

from numpy import asarray
from numpy import savetxt

import geopandas as geop

import twint
import nest_asyncio

import glob
import os

import pymysql

# Crawling Data Twitter

In [26]:
nest_asyncio.apply()

def column_names():
  return twint.output.panda.Tweets_df.columns

def twint_to_pd(columns):
  return twint.output.panda.Tweets_df[columns]

## Ganjar

In [31]:
df_ganjar=pd.read_csv("dataset\ganajar.csv", sep=";")
df_ganjar.head()

,Column1,date,username,tweet,location,label
0,0,27/11/2022 13:36,REdukazi,Pemimpin yang memikirkan rakyat berarti sepert...,ACEH,1
1,1,27/11/2022 13:33,REdukazi,Ganjar Mengerti Keinginan Rakyat #ganjarpran...,ACEH,1
2,2,27/11/2022 13:32,REdukazi,Ganjar Memberikan Kesejahteraan Untuk Para Gur...,ACEH,1
3,3,27/11/2022 13:30,REdukazi,Ganjar Calon Presiden Pilihan Rakyat #ganjar...,ACEH,1
4,4,27/11/2022 11:34,WaktuSumut,Indonesia Butuh Pemimpin Kharismatik seperti G...,ACEH,1


In [32]:
df=df_ganjar[['tweet']]
# df=df.drop_duplicates(subset=['tweet'])
df

,tweet
0,Pemimpin yang memikirkan rakyat berarti sepert...
1,Ganjar Mengerti Keinginan Rakyat #ganjarpran...
2,Ganjar Memberikan Kesejahteraan Untuk Para Gur...
3,Ganjar Calon Presiden Pilihan Rakyat #ganjar...
4,Indonesia Butuh Pemimpin Kharismatik seperti G...
...,...
992,Ganjar Pranowo dipercaya lanjutkan kesuksesan ...
993,Bagi ganjar Pranowo mensejahterakan guru adala...
994,Ganjar Pranowo perintahkan bupati cilacap tak ...
995,Benar apa kata pak Ganjar Pranowo #ganjarprano...


## Anies Baswedan

In [140]:
df_anies=pd.read_csv("dataset/anies.csv", sep=";")
df_anies.head()

,Column1,date,username,tweet,location,label
0,0,27/12/2022 21:36,RiskaRi15138029,@om_eprajol @aniesbaswedan Emng enak ya,ACEH,0
1,1,27/12/2022 19:25,KathyHargrove20,Biadab kelakuan Anies Baswedan penghianat NKRI...,ACEH,0
2,2,27/12/2022 14:28,sitohang76,Padahal @aniesbaswedan sudah dapat nama Johane...,ACEH,0
3,3,27/12/2022 10:12,danialzanif,Solo tolak kedatangan anies baswedan #HTIFPID...,ACEH,0
4,4,27/12/2022 10:11,danialzanif,Anies Baswedan dalang dibalik radikalisme #H...,ACEH,0


In [141]:
df=df_anies.copy()
sum(df["label"]==1)

64

In [142]:
df=df[['tweet', 'label']]
df=df.drop_duplicates(subset=['tweet'])
df

,tweet,label
0,@om_eprajol @aniesbaswedan Emng enak ya,0
1,Biadab kelakuan Anies Baswedan penghianat NKRI...,0
2,Padahal @aniesbaswedan sudah dapat nama Johane...,0
3,Solo tolak kedatangan anies baswedan #HTIFPID...,0
4,Anies Baswedan dalang dibalik radikalisme #H...,0
...,...,...
995,Pergilah jangan ganggu ketenangan Kota Solo Ka...,0
996,Anies kena kartu merah dari masyarakat Solo - ...,0
997,@aniesbaswedan Anies memang keren cadaaass 👍👍 ...,1
998,Curi start kampanye aturan kau langgar anies B...,0


In [145]:
df_anies_label_1=df[df['label']==1] #.iloc[:200]
df_anies_label_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 20 to 997
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   33 non-null     object
 1   label   33 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 792.0+ bytes


In [146]:
df_anies_label_0=df[df['label']==0] #.iloc[:200]
df_anies_label_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   461 non-null    object
 1   label   461 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 10.8+ KB


In [ ]:
df_anies_revisi=pd.concat([df_anies_label_1, df_anies_label_0])
df_anies_revisi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 369
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   320 non-null    object
 1   label   320 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


In [ ]:
import twint
import nest_asyncio

nest_asyncio.apply()
c= twint.Config()
c.Since="2022-01-15"
c.Until="2022-03-15"
c.Search = "Anies Baswedan"
c.Pandas = True
twint.run.Search(c)

In [134]:
def column_names():
  return twint.output.panda.Tweets_df.columns

def twint_to_pd(columns):
  return twint.output.panda.Tweets_df[columns]

In [136]:
data = twint_to_pd(['tweet'])
data.to_csv('anies_revisi.csv')
data.head()

,tweet
0,@AlexThe40661913 @aniesbaswedan @RahmayadiEdy ...
1,@bos_sir @ernawati_sn @aniesbaswedan @AniesInd...
2,@_SEKNAS_RI @aniesbaswedan @RahmayadiEdy @BNPT...
3,"Anies Baswedan - ""Pemimpin menyampaikan pesan,..."
4,@Rizmaya__ @aniesbaswedan Inilah gotbenar hasi...


In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2023-01-01"
    c.Until = "2023-01-16"
    c.Search = "Anies Baswedan"
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    path="hasil_scraping/"
    judul=f"{path}{wilayah}_ANIES.csv"
    data.to_csv(judul, index=False)

## Prabowo

In [22]:
df_prabowo=pd.read_csv("dataset/prabowo.csv", sep=";")
df_prabowo.head()

,username,tweet,label
0,melatiseptian11,Jatim Dianggap Battleground Paling Sengit di P...,1
1,GaniSamudera,“Isu bakal majunya pasangan Prabowo/Khofifah y...,1
2,BaraZhafir,"Isu Disandingkan dengan Khofifah di 2024, Elek...",1
3,Taniaanandaa,"""Kalau beliau jadi Presiden inilah yang akan b...",1
4,mhmmdalbarr,Mobil bantuan Prabowo tersebut akan digunakan ...,1


In [148]:
df=df_prabowo.copy()
sum(df["label"]==1)

952

In [149]:
df=df[['tweet', 'label']]
df=df.drop_duplicates(subset=['tweet'])
df

,tweet,label
0,Jatim Dianggap Battleground Paling Sengit di P...,1
1,“Isu bakal majunya pasangan Prabowo/Khofifah y...,1
2,"Isu Disandingkan dengan Khofifah di 2024, Elek...",1
3,"""Kalau beliau jadi Presiden inilah yang akan b...",1
4,Mobil bantuan Prabowo tersebut akan digunakan ...,1
...,...,...
999,#mendingprabowo #prabowosubianto #prabowopre...,1
1000,#mendingprabowo #prabowosubianto #prabowopre...,1
1001,"Bahkan ada juga, Ketua Majelis Kehormatan Part...",1
1002,Prabowo subianto memang masih belum tergoyahka...,1


In [150]:
df_prabowo_label_1=df[df['label']==1] #.iloc[:200]
df_prabowo_label_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316 entries, 0 to 1003
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   316 non-null    object
 1   label   316 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.4+ KB


In [151]:
df_prabowo_label_0=df[df['label']==0] #.iloc[:200]
df_prabowo_label_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 29 to 945
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   20 non-null     object
 1   label   20 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 480.0+ bytes


In [ ]:
df_anies_revisi=pd.concat([df_anies_label_1, df_anies_label_0])
df_anies_revisi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 369
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   320 non-null    object
 1   label   320 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


## Prabowo

In [167]:
import twint
import nest_asyncio

nest_asyncio.apply()
c= twint.Config()
c.Since="2022-01-15"
c.Until="2022-04-15"
c.Search = "Prabowo Subianto"
c.Pandas = True
twint.run.Search(c)

1514755106905894933 2022-04-15 06:58:54 +0700 <_3angel_> @jimmysinback @prabowo apa ngga cape capres gagal terus? Sepertinya rakyat sudah bosan lihat muka bapak. Baiknya mundur kasih kesempatan wajah baru generasi muda untuk memimpin bangsa ini
1514752831391432707 2022-04-15 06:49:52 +0700 <MenthelRomario> @Nicho_Silalahi @jokowi @prabowo @sandiuno Muka jelek cecurut Niko keknya sebentar lagi stroke nih
1514751281595883526 2022-04-15 06:43:42 +0700 <HeraldSelatan> Prabowo Disebut Bakal Kembali Maju Jadi Calon Presiden, Begini Tanggapan Jubir #calonPresiden #PrabowoSubianto  https://t.co/40GEXZ1xTX
1514747305831804936 2022-04-15 06:27:55 +0700 <Praszken> @ChusnulCh__ @prabowo Wkwkwk mana berani ini orang komen .. wowo oh wowo
1514746843422670854 2022-04-15 06:26:04 +0700 <erwinbm> @ChusnulCh__ @prabowo Biasa bengong mb. Masih ngelirik kaum radikal kek nya
1514745223779262464 2022-04-15 06:19:38 +0700 <IdSinpo> Survei Charta Politika! Elektabilitas Prabowo Subianto Tertinggi Di Kalimanta

In [168]:
def column_names():
  return twint.output.panda.Tweets_df.columns

def twint_to_pd(columns):
  return twint.output.panda.Tweets_df[columns]

In [169]:
data = twint_to_pd(['tweet'])
data.to_csv('prabowo_revisi.csv')
data.tail()

,tweet
4351,@alumni_banten @prabowo @Kemhan_RI Jempol Bu M...
4352,"Catet ya, Menkeu Sri Mulyani menemui Menhan Pr..."
4353,@anisara_rahmah @prabowo @Kemhan_RI Sepakat ba...
4354,RT korantempo: Tiga pasangan calon diprediksi ...
4355,"@yaniahmadandi @prabowo @Kemhan_RI Top banget,..."


In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2023-01-01"
    c.Until = "2023-01-16"
    c.Search = "Prabowo Subianto"
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    path="hasil_scraping/"
    judul=f"{path}{wilayah}_PRABOWO.csv"
    data.to_csv(judul, index=False)

# Modeling

## Ganjar

In [42]:
df_ganjar= pd.concat(map(pd.read_csv, glob.glob(os.path.join('hasil_scraping/', "*_GANJAR.csv"))))
df_ganjar= df_ganjar.reset_index(drop=True)
df_ganjar=df_ganjar.iloc[:1000,0:4]
df_ganjar.tail()
df_ganjar.to_csv("data_ganjar.csv")

In [24]:
df=pd.read_csv("dataset/Dataset.csv", sep=";")
df.head()

,tweet,label
0,Pak @ganjarpranowo ... Sangat memalukan sekali...,0
1,@cahedewe000 @ChusnulCh__ @arsul_sani @ganjarp...,0
2,@amr715882 @gibran_tweet @jokowi @ListyoSigitP...,0
3,@puspita3_eka @aniesbaswedan @Mars_Sahsa @ganj...,0
4,@Catatan_ali7 @amr715882 @gibran_tweet @jokowi...,0


In [43]:
df_ganjar.head()

,date,username,tweet,location
0,2023-01-16 06:33:05,Hendrik66752917,@ganjarpranowo Masih jaman PENCITRAAN bung. Ke...,ACEH
1,2023-01-16 05:49:50,irvan_yonas,@RamliRizal Setidaknya apa yang disampaikan ol...,ACEH
2,2023-01-16 05:22:44,Ariefff92839705,@0512Vip @AndiSinulingga @ganjarpranowo Kamu i...,ACEH
3,2023-01-16 03:02:56,Rey_Annd,@jakapradipta @Irwan2yah1 @DivHumas_Polri @gan...,ACEH
4,2023-01-16 01:09:41,makganjarsumut,Mak Ganjar Sumatera Utara berbagi kebaikan..🇲🇨...,ACEH


In [44]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_ganjar['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal


In [45]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [47]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
# print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  1856 



In [48]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 24, padding='pre')

In [49]:
padding.shape

(1000, 24)

In [50]:
nama_model='ganjar'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

1/1 [==============================] - 0s 315ms/step


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\iki11\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\iki11\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.

## Anies

In [51]:
df_anies= pd.concat(map(pd.read_csv, glob.glob(os.path.join('hasil_scraping/', "*_ANIES.csv"))))
df_anies= df_anies.reset_index(drop=True)
df_anies=df_anies.iloc[:1000,0:4]
df_anies.tail()
df_anies.to_csv("data_anies.csv")

In [83]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_anies['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal


In [84]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [86]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
# print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  1112 



In [87]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')

In [88]:
padding.shape

(1000, 50)

In [89]:
nama_model='anies'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

1/1 [==============================] - 0s 38ms/step


In [60]:
df_anies['Tokoh'] = 'Anies Baswedan'
df_anies['Sentiment'] = result

In [61]:
df_anies.head()

,date,username,tweet,location,Tokoh,Sentiment
0,2022-12-27 21:36:32,RiskaRi15138029,@om_eprajol @aniesbaswedan Emng enak ya,ACEH,Anies Baswedan,1
1,2022-12-27 19:25:44,KathyHargrove20,Biadab kelakuan Anies Baswedan penghianat NKRI...,ACEH,Anies Baswedan,0
2,2022-12-27 14:28:43,sitohang76,Padahal @aniesbaswedan sudah dapat nama Johane...,ACEH,Anies Baswedan,0
3,2022-12-27 10:12:08,danialzanif,Solo tolak kedatangan anies baswedan #HTIFPID...,ACEH,Anies Baswedan,0
4,2022-12-27 10:11:09,danialzanif,Anies Baswedan dalang dibalik radikalisme #H...,ACEH,Anies Baswedan,0


In [67]:
def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 # password='f#Ur8J3N',
                                 database='klasifikasi_sentimen')
    # create cursor
    cursor=connection.cursor()
    
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

In [68]:
insert_to_list_rekomen_db(rp2=df_anies)

## Prabowo

In [94]:
df_prabowo= pd.concat(map(pd.read_csv, glob.glob(os.path.join('hasil_scraping/', "*_PRABOWO.csv"))))
df_prabowo= df_prabowo.reset_index(drop=True)
df_prabowo=df_prabowo.iloc[:1000,0:4]
df_prabowo.tail()
df_prabowo.to_csv("data_prabowo.csv")

In [95]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_prabowo['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal


In [96]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [98]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
# print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  1689 



In [99]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 34, padding='pre')

In [100]:
padding.shape

(1000, 34)

In [101]:
nama_model='prabowo'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\iki11\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\iki11\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\Materi Kuliah\SKRIPSI\skripsiSentiment\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.

# Set Up Data GeoJson Daerah Indonesia tiap Provinsi

In [ ]:
df=geop.read_file("indonesia-prov.geojson")
df.head()

,ID,kode,Propinsi,SUMBER,geometry
0,2,52,NUSATENGGARA BARAT,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((117.62720 -8.50640, 117.63470 ..."
1,3,75,GORONTALO,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((122.18814 1.04530, 122.22627 1.00335..."
2,4,74,SULAWESI TENGGARA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((120.98423 -2.83534, 121.07834 ..."
3,5,34,DAERAH ISTIMEWA YOGYAKARTA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((110.01183 -7.88690, 110.04295 -7.892..."
4,6,33,JAWA TENGAH,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((108.82934 -6.74608, 108.85489 ..."


In [ ]:
df.Propinsi.to_list()

In [ ]:
#mengganti value Status
replacement_mapping_dict = {
    "DI. ACEH" : "ACEH",
    "NUSATENGGARA BARAT" : "NUSA TENGGARA BARAT",
    "DAERAH ISTIMEWA YOGYAKARTA" : "DI YOGYAKARTA",
    "BANGKA BELITUNG" : "KEPULAUAN BANGKA BELITUNG"
}
df["Propinsi"] = df["Propinsi"].replace(replacement_mapping_dict)
df.head()

,ID,kode,Propinsi,SUMBER,geometry
0,2,52,NUSA TENGGARA BARAT,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((117.62720 -8.50640, 117.63470 ..."
1,3,75,GORONTALO,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((122.18814 1.04530, 122.22627 1.00335..."
2,4,74,SULAWESI TENGGARA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((120.98423 -2.83534, 121.07834 ..."
3,5,34,DI YOGYAKARTA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((110.01183 -7.88690, 110.04295 -7.892..."
4,6,33,JAWA TENGAH,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((108.82934 -6.74608, 108.85489 ..."


In [ ]:
nc_counties = df.to_crs(4326)
nc_counties['lon_centroid'] = nc_counties.centroid.x  
nc_counties['lat_centroid'] = nc_counties.centroid.y

C:\Users\iki11\AppData\Local\Temp\ipykernel_17112\1284419102.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nc_counties['lon_centroid'] = nc_counties.centroid.x
C:\Users\iki11\AppData\Local\Temp\ipykernel_17112\1284419102.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nc_counties['lat_centroid'] = nc_counties.centroid.y


In [ ]:
nc_counties.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID            34 non-null     int64   
 1   kode          34 non-null     int64   
 2   Propinsi      34 non-null     object  
 3   SUMBER        34 non-null     object  
 4   geometry      34 non-null     geometry
 5   lon_centroid  34 non-null     float64 
 6   lat_centroid  34 non-null     float64 
dtypes: float64(2), geometry(1), int64(2), object(2)
memory usage: 2.0+ KB


# Set Up Data Luas Wilayah

In [ ]:
headers=['Propinsi', 'Ibu Kota Wilayah','Luas Wilayah (km2)', 'Persentase Terhadap Luas Wilayah', 'Jumlah Pulau']
luas_wilayah=pd.read_excel("Luas Daerah dan Jumlah Pulau Menurut Provinsi, 2021.xlsx", names=headers, header=None)
luas_wilayah.head()

,Propinsi,Ibu Kota Wilayah,Luas Wilayah (km2),Persentase Terhadap Luas Wilayah,Jumlah Pulau
0,"Luas Daerah dan Jumlah Pulau Menurut Provinsi,...",NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Provinsi\n Province,Ibu Kota Wilayah,Luas Wilayah (km2),Persentase Terhadap Luas Wilayah,Jumlah Pulau
3,Aceh,Banda Aceh,"57 956,00","3,02",363
4,Sumatera Utara,Medan,"72 981,23","3,81",229


In [ ]:
data = luas_wilayah.drop(luas_wilayah.index[[0,1,2]]).reset_index()
data = data.drop(['index', 'Ibu Kota Wilayah','Persentase Terhadap Luas Wilayah','Jumlah Pulau'], axis=1)
data = data.dropna() 
data['Propinsi'] = data['Propinsi'].str.upper()
data.head()

,Propinsi,Luas Wilayah (km2)
0,ACEH,"57 956,00"
1,SUMATERA UTARA,"72 981,23"
2,SUMATERA BARAT,"42 012,89"
3,RIAU,"87 023,66"
4,JAMBI,"50 058,16"


In [ ]:
radius=data["Luas Wilayah (km2)"].values.tolist()
radius

['57 956,00',
 '72 981,23',
 '42 012,89',
 '87 023,66',
 '50 058,16',
 '91 592,43',
 '19 919,33',
 '34 623,80',
 '16 424,06',
 '8 201,72',
 '664,01',
 '35 377,76',
 '32 800,69',
 '3 133,15',
 '47 803,49',
 '9 662,92',
 '5 780,06',
 '18 572,32',
 '48 718,10',
 '147 307,00',
 '153 564,50',
 '38 744,23',
 '129 066,64',
 '75 467,70',
 '13 892,47',
 '61 841,29',
 '46 717,48',
 '38 067,70',
 '11 257,07',
 '16 787,18',
 '46 914,03',
 '31 982,50',
 '102 955,15',
 '319 036,05',
 '1 916 906,77']

In [ ]:
radius_fix=[]
for i in range(len(radius)):
    x=radius[i].replace(" ", "").rstrip(radius[i][-3:])
    x=round(int(x)/2)
    radius_fix.append(x)
radius_fix

[28978,
 36490,
 21006,
 43512,
 25029,
 45796,
 9960,
 17312,
 8212,
 4100,
 332,
 176,
 16400,
 1566,
 23902,
 483,
 289,
 928,
 24359,
 73654,
 76782,
 19372,
 645,
 3773,
 6946,
 30920,
 23358,
 1903,
 562,
 8394,
 23457,
 15991,
 514,
 159518,
 958453]

In [ ]:
data['radius']=radius_fix
data.drop(data[data['Propinsi'] == 'INDONESIA'].index, inplace = True)
data.head()

,Propinsi,Luas Wilayah (km2),radius
0,ACEH,"57 956,00",28978
1,SUMATERA UTARA,"72 981,23",36490
2,SUMATERA BARAT,"42 012,89",21006
3,RIAU,"87 023,66",43512
4,JAMBI,"50 058,16",25029


# Merge Data

In [ ]:
outer_merged = pd.merge(
    nc_counties, data, how="right", on=["Propinsi"]
)

In [ ]:
outer_merged.head()

,ID,kode,Propinsi,SUMBER,geometry,lon_centroid,lat_centroid,Luas Wilayah (km2),radius
0,32,11,ACEH,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((98.25574 4.29095, 98.14540 4.2...",96.913918,4.226786,"57 956,00",28978
1,24,12,SUMATERA UTARA,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((98.25574 4.29095, 98.25168 4.2...",99.066609,2.188556,"72 981,23",36490
2,26,13,SUMATERA BARAT,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((99.16198 0.23181, 99.23553 0.3...",100.459521,-0.862727,"42 012,89",21006
3,19,14,RIAU,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"MULTIPOLYGON (((101.91405 -0.92096, 101.84528 ...",101.804084,0.503161,"87 023,66",43512
4,29,15,JAMBI,Peta Dasar BAKOSURTANAL Skala 1 : 250.000,"POLYGON ((101.29416 -2.28020, 101.30608 -2.238...",102.725336,-1.696863,"50 058,16",25029


In [ ]:
parameter_geo = outer_merged.drop(['ID', 'kode','SUMBER','geometry', 'Luas Wilayah (km2)'], axis=1)
parameter_geo.head()

,Propinsi,lon_centroid,lat_centroid,radius
0,ACEH,96.913918,4.226786,28978
1,SUMATERA UTARA,99.066609,2.188556,36490
2,SUMATERA BARAT,100.459521,-0.862727,21006
3,RIAU,101.804084,0.503161,43512
4,JAMBI,102.725336,-1.696863,25029


In [ ]:
parameter_geo.to_csv("parameter_geo.csv")

In [ ]:
parameter_geo.Propinsi[0]

'ACEH'

In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2021-01-01"
    c.Until = "2021-05-28"
    c.Search = "Ganjar Pranowo"
    c.Limit=1000
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    judul=f"{wilayah}_JAN_to_MEI_2021.csv"
    data.to_csv(judul, index=False)

In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2021-06-01"
    c.Until = "2021-12-28"
    c.Search = "Ganjar Pranowo"
    c.Limit=1000
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    judul=f"{wilayah}_JUN_to_DES_2021.csv"
    data.to_csv(judul, index=False)

In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2022-01-01"
    c.Until = "2022-05-31"
    c.Search = "Ganjar Pranowo"
    
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    judul=f"{wilayah}_JAN_to_MEI_2022.csv"
    data.to_csv(judul, index=False)

In [ ]:
for i in range(len(parameter_geo)):
    wilayah= parameter_geo.Propinsi[i]
    latitude=parameter_geo.lat_centroid[i]
    longitude=parameter_geo.lon_centroid[i]
    radius=parameter_geo.radius[i]
    param=f"{latitude},{longitude},{radius}"+"km"
    c= twint.Config()
    c.Since = "2022-06-01"
    c.Until = "2022-12-28"
    c.Search = "Ganjar Pranowo"
    c.Geo=param
    c.Pandas = True
    twint.run.Search(c)
    data = twint_to_pd(['date','username','tweet'])
    data["location"] = wilayah
    judul=f"{wilayah}_JUN_to_DES_2022.csv"
    data.to_csv(judul, index=False)

In [50]:
import twint
import nest_asyncio

nest_asyncio.apply()
c= twint.Config()
c.Until="2023-01-15"
c.Search = "Ganjar Pranowo"
c.Pandas = True
twint.run.Search(c)

# def scraping(tokoh, wilayah, latitude, longitude, radius, tanggal_awal, tanggal_akhir):
#     nest_asyncio.apply()
#     c= twint.Config()
#     c.Since=tanggal_awal
#     c.Until=tanggal_akhir
    
    

1614306196298821632 2023-01-14 23:59:24 +0700 <SugengSpriyadi> @tony60631622 @ridwankamil Keren nih pasti suruhannya @ganjarpranowo yang kemiskinannya terburuk di pulau jawa sama kemaren banjir parah banget+korban terorisme terbanyak kan dijateng 🫢
1614305875094552576 2023-01-14 23:58:07 +0700 <KristyH56705066> @GanjaranApp @ganjarpranowo dalam menangani banjir memang pak ganajr ini gercep yahh
1614305125480423427 2023-01-14 23:55:08 +0700 <kurakurabrsayap> @FerCZ18 @ganjarpranowo @aniesbaswedan Musim hujan loh 🤣🤣  https://t.co/miQN5QtlC9
1614305083407355906 2023-01-14 23:54:58 +0700 <rass_aa> @kakekhalal2 @DiniHrdianti Pak @ganjarpranowo kok gak di senggol2, Yo Bu Mega...
1614304666078302209 2023-01-14 23:53:19 +0700 <DahonoB> @Fahrihamzah @partaigeloraid Anies B bisa menang Pilpres 2024 Jika Ganjar Pranowo tidak ikut Pilpres 2024  https://t.co/qK9lpZkryW
1614304032264433664 2023-01-14 23:50:48 +0700 <4_gp2024> dan tanggul di Juwana harus segera dilakukan tahun ini.  Ganjar sudah memi

In [51]:
twint.output.panda.Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [52]:
if len(twint.output.panda.Tweets_df.columns) == 0:
    print("None Tweets")
else:
    print("Tweets")

Tweets


In [2]:
def column_names():
  return twint.output.panda.Tweets_df.columns

def twint_to_pd(columns):
  return twint.output.panda.Tweets_df[columns]

In [5]:
data = twint_to_pd(['date','username','tweet', 'place'])
data.head()

,date,username,tweet,place
0,2023-01-14 23:59:24,SugengSpriyadi,@tony60631622 @ridwankamil Keren nih pasti sur...,
1,2023-01-14 23:58:07,KristyH56705066,@GanjaranApp @ganjarpranowo dalam menangani ba...,
2,2023-01-14 23:55:08,kurakurabrsayap,@FerCZ18 @ganjarpranowo @aniesbaswedan Musim h...,
3,2023-01-14 23:54:58,rass_aa,@kakekhalal2 @DiniHrdianti Pak @ganjarpranowo ...,
4,2023-01-14 23:53:19,DahonoB,@Fahrihamzah @partaigeloraid Anies B bisa mena...,


In [10]:
data.to_csv("prabowo.csv", index=False)

In [27]:
twint.output.panda.Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [28]:
data = twint_to_pd(['date','username','tweet'])
wilayah="JATENG"
data["location"] = wilayah
data.tail()

C:\Users\iki11\AppData\Local\Temp\ipykernel_2612\233969598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["location"] = wilayah


,date,username,tweet,location
15,2021-02-27 11:11:20,felixtigrism,@WagimanDeep212_ @ganjarpranowo Info mam minum...,JATENG
16,2021-02-27 11:05:09,ardiyantosecang,@ganjarpranowo Mudah mudahan Semarang kota beb...,JATENG
17,2021-02-27 11:04:17,ardiyantosecang,@ganjarpranowo Mudah mudahan segera dimulai.,JATENG
18,2021-02-27 10:30:24,bocahmluis,"Assalamualaikum, Ijin melaporkan ndoro jembata...",JATENG
19,2021-02-27 08:01:50,nashih_ismy,"@ganjarpranowo siap menunggu hasilnya, Ndoro.....",JATENG


In [125]:
path="hasil_scraping/"
judul=f"{path}{wilayah}_JAN_to_FEB_2021.csv"
data.to_csv(judul, index=False)